In [ ]:
from collections import deque
import random

class FilaExamesDiarios:
    def __init__(self):
        self._fila = deque()
    def receber_exame_na_triagem(self, exame):
        self._fila.append(exame)
    def encaminhar_para_dimensao(self):
        if not self._fila:
            return None
        return self._fila.popleft()
    def priorizar_reprocesso(self, exame):
        self._fila.appendleft(exame)
    def __len__(self):
        return len(self._fila)

class PilhaConsultasRecentes:
    def __init__(self):
        self._itens = []
    def empilhar(self, x):
        self._itens.append(x)
    def desempilhar(self):
        return self._itens.pop() if self._itens else None
    def topo(self):
        return self._itens[-1] if self._itens else None
    def __len__(self):
        return len(self._itens)

class PilhaDesfazerProcessamento:
    def __init__(self):
        self._itens = []
    def empilhar(self, x):
        self._itens.append(x)
    def desempilhar(self):
        return self._itens.pop() if self._itens else None
    def topo(self):
        return self._itens[-1] if self._itens else None
    def __len__(self):
        return len(self._itens)

def estimar_dimensao_lesao(id_imagem, semente=0):
    random.seed(id_imagem + semente)
    return round(random.uniform(0.5, 12.0), 2)

def gerar_laudo_dimensional(exame):
    medida = estimar_dimensao_lesao(exame["id"])
    return {"id": exame["id"], "paciente": exame["paciente"], "tipo": exame["tipo"], "medida_mm": medida, "laudo": f"Medida estimada: {medida} mm"}

def localizar_laudo_por_id_sequencial(repositorio_laudos, id_laudo):
    for i, item in enumerate(repositorio_laudos):
        if item["id"] == id_laudo:
            return i
    return -1

def organizar_laudos_por_id(laudos):
    if len(laudos) <= 1:
        return laudos
    meio = len(laudos) // 2
    esquerda = organizar_laudos_por_id(laudos[:meio])
    direita = organizar_laudos_por_id(laudos[meio:])
    i = j = 0
    mesclado = []
    while i < len(esquerda) and j < len(direita):
        if esquerda[i]["id"] <= direita[j]["id"]:
            mesclado.append(esquerda[i]); i += 1
        else:
            mesclado.append(direita[j]); j += 1
    while i < len(esquerda):
        mesclado.append(esquerda[i]); i += 1
    while j < len(direita):
        mesclado.append(direita[j]); j += 1
    return mesclado

def localizar_laudo_por_id_binaria(laudos_ordenados, id_laudo):
    inicio, fim = 0, len(laudos_ordenados) - 1
    while inicio <= fim:
        meio = (inicio + fim) // 2
        id_atual = laudos_ordenados[meio]["id"]
        if id_atual == id_laudo:
            return meio
        if id_atual < id_laudo:
            inicio = meio + 1
        else:
            fim = meio - 1
    return -1

def registrar_consulta_de_laudo(repositorio_laudos, pilha_consultas, id_laudo, metodo_busca="sequencial"):
    alvo = None
    if metodo_busca == "binaria":
        ordenados = organizar_laudos_por_id(repositorio_laudos)
        posicao = localizar_laudo_por_id_binaria(ordenados, id_laudo)
        if posicao != -1:
            alvo = ordenados[posicao]
    else:
        posicao = localizar_laudo_por_id_sequencial(repositorio_laudos, id_laudo)
        if posicao != -1:
            alvo = repositorio_laudos[posicao]
    if alvo:
        pilha_consultas.empilhar(alvo)
        return alvo
    return None

def desfazer_ultimo_laudo_emitido(fila, repositorio_laudos, pilha_desfazer):
    topo = pilha_desfazer.desempilhar()
    if not topo:
        return False
    acao, laudo = topo
    if acao == "emitido":
        i = localizar_laudo_por_id_sequencial(repositorio_laudos, laudo["id"])
        if i != -1:
            removido = repositorio_laudos.pop(i)
            fila.priorizar_reprocesso({"id": removido["id"], "paciente": removido["paciente"], "tipo": removido["tipo"], "imagem": f"imagem_{removido['id']}"})
            return True
    return False

def carregar_lote_exames_do_dia():
    return [
        {"id": 13, "paciente": "Ana", "tipo": "Dermato", "imagem": "imagem_13"},
        {"id": 11, "paciente": "Bruno", "tipo": "Dermato", "imagem": "imagem_11"},
        {"id": 19, "paciente": "Clara", "tipo": "Dermato", "imagem": "imagem_19"},
        {"id": 17, "paciente": "Diego", "tipo": "Dermato", "imagem": "imagem_17"},
        {"id": 11, "paciente": "Eva", "tipo": "Dermato", "imagem": "imagem_11"},
    ]

def formatar_linha_laudo(laudo):
    return f"{laudo['id']:>4}  | {laudo['paciente']:<12} | {laudo['tipo']:<9} | {laudo['medida_mm']:>10.2f}"

def imprimir_tabela_laudos(laudos):
    if not laudos:
        print("Nenhum laudo emitido até o momento.")
        return
    cabecalho = " ID   | Paciente     | Tipo      | Medida (mm)"
    linha = "-" * len(cabecalho)
    print(cabecalho)
    print(linha)
    for l in laudos:
        print(formatar_linha_laudo(l))

fila = FilaExamesDiarios()
pilha_consultas = PilhaConsultasRecentes()
pilha_desfazer = PilhaDesfazerProcessamento()
repositorio_laudos = []

for exame in carregar_lote_exames_do_dia():
    fila.receber_exame_na_triagem(exame)

def existe_laudo_emitido():
    return len(repositorio_laudos) > 0

def existe_acao_para_desfazer():
    return len(pilha_desfazer) > 0

def existem_consultas_registradas():
    return len(pilha_consultas) > 0

def ids_disponiveis_para_consulta():
    return [l["id"] for l in organizar_laudos_por_id(repositorio_laudos)]

def menu_operacional():
    print("\n=== Laboratório de Patologia DASA • Dimensionamento de Tecidos ===")
    print(f"Fila: {len(fila)} | Laudos: {len(repositorio_laudos)} | Desfazer: {len(pilha_desfazer)} | Consultas recentes: {len(pilha_consultas)}")
    print("1) Encaminhar próximo exame e emitir laudo")
    print("2) Consultar laudo por ID (sequencial)")
    print("3) Consultar laudo por ID (binária)")
    print("4) Listar laudos")
    print("5) Desfazer último laudo emitido")
    print("6) Ver topo da pilha de consultas")
    print("0) Sair")

while True:
    menu_operacional()
    opcao = input("Escolha: ").strip()
    print(f"\nVocê digitou a opção {opcao}.")
    if opcao == "1":
        exame_encaminhado = fila.encaminhar_para_dimensao()
        if not exame_encaminhado:
            print("Sem exames na fila.")
        else:
            laudo = gerar_laudo_dimensional(exame_encaminhado)
            repositorio_laudos.append(laudo)
            pilha_desfazer.empilhar(("emitido", laudo))
            print("Laudo emitido:")
            imprimir_tabela_laudos([laudo])
    elif opcao == "2":
        if not existe_laudo_emitido():
            print("Não há laudos emitidos. Use a opção 1 primeiro.")
            continue
        print("Laudos disponíveis para consulta (IDs):", ids_disponiveis_para_consulta())
        try:
            id_laudo_busca = int(input("ID do laudo: ").strip())
        except:
            print("ID inválido.")
            continue
        resultado = registrar_consulta_de_laudo(repositorio_laudos, pilha_consultas, id_laudo_busca, metodo_busca="sequencial")
        if resultado:
            print("Laudo localizado:")
            imprimir_tabela_laudos([resultado])
        else:
            print("Laudo não encontrado. Use a opção 4 para listar os laudos e ver os IDs disponíveis.")
    elif opcao == "3":
        if not existe_laudo_emitido():
            print("Não há laudos emitidos. Use a opção 1 primeiro.")
            continue
        print("Laudos disponíveis para consulta (IDs):", ids_disponiveis_para_consulta())
        try:
            id_laudo_busca = int(input("ID do laudo: ").strip())
        except:
            print("ID inválido.")
            continue
        resultado = registrar_consulta_de_laudo(repositorio_laudos, pilha_consultas, id_laudo_busca, metodo_busca="binaria")
        if resultado:
            print("Laudo localizado:")
            imprimir_tabela_laudos([resultado])
        else:
            print("Laudo não encontrado. Use a opção 4 para listar os laudos e ver os IDs disponíveis.")
    elif opcao == "4":
        if not existe_laudo_emitido():
            print("Não há laudos emitidos. Use a opção 1 primeiro.")
            continue
        print("Laudos emitidos:")
        imprimir_tabela_laudos(organizar_laudos_por_id(repositorio_laudos))
    elif opcao == "5":
        if not existe_acao_para_desfazer():
            print("Não há nada para se desfazer.")
            continue
        desfeito_com_sucesso = desfazer_ultimo_laudo_emitido(fila, repositorio_laudos, pilha_desfazer)
        if desfeito_com_sucesso:
            print("A última emissão de laudo foi desfeita e o exame voltou para o início da fila.")
        else:
            print("Não foi possível desfazer a última emissão.")
    elif opcao == "6":
        if not existem_consultas_registradas():
            print("Nenhuma consulta registrada ainda.")
            continue
        print("Último laudo consultado:")
        imprimir_tabela_laudos([pilha_consultas.topo()])
    elif opcao == "0":
        print("Sessão encerrada.")
        break
    else:
        print("Opção inválida!")